In [1]:
import random as rnd
import pandas as pd
import numpy as np
from copy import deepcopy

In [2]:
#Формируем псевдо имена студентов
import string


def student_name(length):
    letters = string.ascii_lowercase
    random_name = ''.join(rnd.choice(letters) for i in range(length))
    return random_name

In [3]:
# генерируем данные
def generate_data(stud:int,uni:int):
    students=[]
    companies=[]
    stud_points={}
    for i in range(1,stud+1):
        students.append('student#'+student_name(5))
    for j in range(1,uni+1):
        companies.append('company#'+str(j))
    for s in students:
        stud_points[s]=[]
        for it in range(5): #сгенерируем баллы, чтобы предпочтения компаний были обоснованными
            stud_points[s].append(int(rnd.triangular(50,100,70)))
    return students,companies,stud_points

In [4]:
A,C,exam_points=generate_data(350,10)

In [5]:
pref_stud_dict={}
quota={} #количество мест для стажировки
student_points=dict.fromkeys(A)

for i in A:
    pref_stud_dict[i]=rnd.sample(C,7)
    
for u in C:
    quota[u]=rnd.randint(10,50)
    
for s in A:
    for u in C:
        if u in pref_stud_dict[s]:
            points_for_company={}
            points_for_company[u]=sum(rnd.sample(exam_points[s],3))
            if student_points[s]:
                student_points[s][u]=points_for_company[u]
            else:
                student_points[s]=points_for_company

In [6]:
pref_company_dict=dict.fromkeys(C)
for u in C:
    for s in A:
        if u in pref_stud_dict[s]:
            if pref_company_dict[u]:
                pref_company_dict[u][s]=student_points[s][u]
            else:
                pref_company_dict[u]={s:student_points[s][u]}
                
for i in pref_company_dict.keys():
    pref_company_dict[i]=sorted(pref_company_dict[i],key=pref_company_dict[i].get,reverse=True)

In [7]:
def insert_pos(pref_comp,comp_match,stud):
    pos=0
    m=deepcopy(comp_match)
    for v in pref_comp:
        if v==stud or (not m):
            break
        elif v==m[0]:
            pos+=1
            m.pop(0)
    comp_match.insert(pos,stud)

def stable_matching(stud_prefs, company_prefs, quota): # на вход подаю словари с предпочтениями
    student_picks=deepcopy(stud_prefs)
    company_picks=deepcopy(company_prefs)
    students = list(student_picks.keys()) # сформируем список студентов, чтобы потом по нему проходиться
    student_matching = dict.fromkeys(student_picks.keys()) # распределение - модель со стороны студентов
    company_matching = {c: [] for c in company_picks.keys()} # распределение - модель со стороны компаний
    points_to_enter={c:None for c in company_picks.keys()}

    while students: #пока не пустой - цикл работает
        a = students.pop(0) # берем студента для расммотрения
        a_i = student_picks[a] # берем список предпочтений студента
        while a_i and (not student_matching[a]): 
            if a not in company_picks[a_i[0]]:
                a_i.remove(a_i[0])
            else:
                c = a_i[0] # первый вуз в списке предпочтений студента
                c_matches = company_matching[c] # список предварительно попавших на стажировку
                insert_pos(company_picks[c], c_matches, a)
                student_matching[a] = c
                if quota[c]<len(c_matches):
                    worst_match = c_matches.pop()
                    student_matching[worst_match] = None
                    student_picks[worst_match].remove(c)
                    company_picks[c].remove(worst_match)
                    students.append(worst_match)
                    
                    
    not_entered=[a for a,c in student_matching.items() if c==None] #формируем список непоступивших

    return student_matching, company_matching, not_entered

In [8]:
student_matches, company_matches,not_entered= stable_matching(pref_stud_dict, pref_company_dict, quota)

In [9]:
student_matches

{'student#bhxky': 'company#2',
 'student#nmtqx': 'company#4',
 'student#zshjb': 'company#2',
 'student#evbgd': 'company#6',
 'student#faqtb': 'company#4',
 'student#sbzsv': 'company#6',
 'student#rdtgj': 'company#8',
 'student#pfddh': 'company#1',
 'student#iwwjv': 'company#3',
 'student#mhpjx': 'company#2',
 'student#hgauv': 'company#9',
 'student#vamly': 'company#5',
 'student#flywn': 'company#6',
 'student#ayvqq': 'company#5',
 'student#tesfz': 'company#1',
 'student#sjpza': 'company#8',
 'student#adhkz': 'company#5',
 'student#fzqsa': 'company#3',
 'student#yfwol': 'company#6',
 'student#bydmg': 'company#10',
 'student#qjqln': 'company#8',
 'student#lxnfs': 'company#9',
 'student#jhlgx': 'company#9',
 'student#ymlou': 'company#3',
 'student#ftauh': 'company#10',
 'student#ktagr': 'company#6',
 'student#mmvqw': 'company#1',
 'student#vrubc': 'company#7',
 'student#elbgi': 'company#6',
 'student#gysvs': 'company#9',
 'student#zflge': 'company#8',
 'student#kufjd': 'company#3',
 'stud

In [10]:
company_matches

{'company#1': ['student#kmlht',
  'student#mmvqw',
  'student#yjsow',
  'student#tesfz',
  'student#rauqs',
  'student#qzwrj',
  'student#pfddh',
  'student#xjqjc',
  'student#zbims',
  'student#gluxy',
  'student#kxahu',
  'student#ttcpy',
  'student#alqtr',
  'student#arkyz',
  'student#fqnqe',
  'student#ksouw',
  'student#opqpm',
  'student#xryyg',
  'student#qxnwc',
  'student#jbwpq',
  'student#yqckp',
  'student#jwxqh',
  'student#xcafm',
  'student#odavi',
  'student#mohye',
  'student#xqnpi',
  'student#hsvlj',
  'student#cekvq',
  'student#emuzz',
  'student#vjrzh',
  'student#emfcl',
  'student#vpjiq',
  'student#gmohn'],
 'company#2': ['student#gufxf',
  'student#jahdw',
  'student#gwfys',
  'student#ftcrd',
  'student#ntrul',
  'student#xaloa',
  'student#gxdap',
  'student#hgoyj',
  'student#zshjb',
  'student#sgphk',
  'student#qsftk',
  'student#adkdl',
  'student#kknex',
  'student#yjohp',
  'student#plvox',
  'student#llvsr',
  'student#ydnpy',
  'student#mhpjx',
  's